In [1]:
# ============================================================
# 📦 Feature Engineering (FE)
# ============================================================

import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

BASE_DIR = Path.cwd().resolve()
CLEAN_PATH = BASE_DIR.parents[1] / "data" / "interim" / "train_clean.csv"
MONTHLY_PATH = BASE_DIR.parents[1] / "data" / "processed" / "train_monthly.csv"

print(f"Loading cleaned data...")
df = pd.read_csv(CLEAN_PATH)
monthly = pd.read_csv(MONTHLY_PATH)
print("✔ Loaded:", df.shape, monthly.shape)

Loading cleaned data...
✔ Loaded: (10535, 9) (3734, 8)


In [4]:
monthly["hs2"] = monthly["hs4"].astype(str).str[:2]
monthly["hs3"] = monthly["hs4"].astype(str).str[:3]

monthly["ym"] = pd.to_datetime(
    monthly["year"].astype(str) + "-" + monthly["month"].astype(str).str.zfill(2)
)


# ------------------------------------------------------------
# 🧹 (추가) type 컬럼 제거 — 모든 값이 1이므로 필요 없음
# ------------------------------------------------------------
if "type" in monthly.columns:
    monthly = monthly.drop(columns=["type"])
if "type" in df.columns:
    df = df.drop(columns=["type"])


In [5]:

# ============================================================
# 🗓 1) YM 생성
# ============================================================
monthly["ym"] = pd.to_datetime(
    monthly["year"].astype(str) + "-" + monthly["month"].astype(str).str.zfill(2)
)
df["ym"] = pd.to_datetime(
    df["year"].astype(str) + "-" + df["month"].astype(str).str.zfill(2)
)


# ============================================================
# 📌 2) pivot_item / pivot_hs3 생성
# ============================================================
pivot_item = monthly.pivot(index="item_id", columns="ym", values="total_value").fillna(0)

# hs3_total은 item 여러 개를 합산해야 하므로 별도 집계
monthly_hs3 = (
    monthly.groupby(["hs3", "ym"], as_index=False)["total_value"].sum()
)

pivot_hs3 = (
    monthly_hs3
    .pivot(index="hs3", columns="ym", values="total_value")
    .fillna(0)
)

print("pivot_item:", pivot_item.shape)
print("pivot_hs3:", pivot_hs3.shape)


pivot_item: (100, 43)
pivot_hs3: (54, 43)


In [6]:
# ============================================================
# 🧮 3) zero_ratio, volatility(CV)
# ============================================================

def compute_zero_vol(series):
    arr = series.values.astype(float)
    zero_ratio = (arr == 0).mean()
    vol = arr.std()
    cv = (arr.std() / (arr.mean() + 1e-9))
    return zero_ratio, vol, cv

records = []
for item in pivot_item.index:
    zr, vol, cv = compute_zero_vol(pivot_item.loc[item])
    records.append([item, zr, vol, cv])

meta = pd.DataFrame(records, columns=["item_id", "zero_ratio", "volatility", "cv"])

print("✔ zero_ratio / volatility 계산 완료:", meta.shape)



✔ zero_ratio / volatility 계산 완료: (100, 4)


In [7]:
# ============================================================
# 🔗 4) HS3_total best lag 1~3 계산
# ============================================================

def best_hs3_lag(item_series, hs3_series_all, max_lag=6):
    best = []
    for lag in range(1, max_lag+1):
        for hs3 in hs3_series_all.index:
            hs3_s = hs3_series_all.loc[hs3].values.astype(float)
            if len(item_series) <= lag:
                continue
            corr = np.corrcoef(hs3_s[:-lag], item_series[lag:])[0,1]
            best.append((hs3, lag, corr))
    df = pd.DataFrame(best, columns=["hs3", "lag", "corr"])
    df = df.sort_values("corr", ascending=False).head(3)
    return df

hs3_lag_records = []

for item in pivot_item.index:
    series_item = pivot_item.loc[item]
    top3 = best_hs3_lag(series_item, pivot_hs3)
    top3["item_id"] = item
    hs3_lag_records.append(top3)

hs3_lag_df = pd.concat(hs3_lag_records, ignore_index=True)
print("✔ HS3_total best lag top3 생성 완료")




✔ HS3_total best lag top3 생성 완료


In [8]:
# ============================================================
# 🔗 5) HS3 → HS4 lead-follow (single best leader)
# ============================================================

pairs_hs3 = []

for hs3 in pivot_hs3.index:
    leader_s = pivot_hs3.loc[hs3].values.astype(float)
    for item in pivot_item.index:
        follower_s = pivot_item.loc[item].values.astype(float)
        best_lag = None
        best_corr = -999

        for lag in range(1, 7):
            if len(leader_s) <= lag:
                continue
            corr = np.corrcoef(leader_s[:-lag], follower_s[lag:])[0,1]
            if corr > best_corr:
                best_corr = corr
                best_lag = lag
        
        pairs_hs3.append([hs3, item, best_lag, best_corr])

pairs_hs3 = pd.DataFrame(
    pairs_hs3, columns=["lead_hs3", "follow_item", "best_lag_hs3", "corr_hs3"]
)

# follower별 single best leader만 남기기
pairs_hs3_best = (
    pairs_hs3.sort_values("corr_hs3", ascending=False)
             .groupby("follow_item").head(1)
)

print("✔ HS3→HS4 single best leader lag 계산 완료")




✔ HS3→HS4 single best leader lag 계산 완료


In [9]:
# ============================================================
# 🔗 6) HS4 multi-leader (top3 weighted)
# ============================================================

# 우선 item_id 단위 pair 계산부터 해야 함
# baseline 방식 그대로 적용

def find_pairs_item(pivot, max_lag=6, corr_threshold=0.2):
    items = pivot.index.to_list()
    months = pivot.columns
    n_months = len(months)
    res = []

    for A in items:
        x = pivot.loc[A].values.astype(float)
        if np.count_nonzero(x) < 6: 
            continue

        for B in items:
            if A == B:
                continue
            y = pivot.loc[B].values.astype(float)
            if np.count_nonzero(y) < 6:
                continue

            best_corr = -999
            best_lag = None

            for lag in range(1, max_lag+1):
                if n_months <= lag:
                    continue
                corr = np.corrcoef(x[:-lag], y[lag:])[0,1]
                if corr > best_corr:
                    best_corr = corr
                    best_lag = lag

            if best_corr >= corr_threshold:
                res.append([A, B, best_lag, best_corr])
    return pd.DataFrame(res, columns=["leader", "follower", "lag", "corr"])


pairs_hs4 = find_pairs_item(pivot_item, corr_threshold=0.3)

# follower별 top3
top3_hs4 = (
    pairs_hs4.sort_values("corr", ascending=False)
             .groupby("follower").head(3)
)

# weighted lag (corr 가중 평균)
def weighted_lag(df):
    w = df["corr"]
    return np.average(df["lag"], weights=w)

multi_leader_lag = (
    top3_hs4.groupby("follower")
            .apply(lambda x: weighted_lag(x))
            .reset_index(name="multi_leader_lag")
)

print("✔ HS4 multi-leader weighted lag 계산 완료")




✔ HS4 multi-leader weighted lag 계산 완료


In [10]:
# ============================================================
# 📦 7) 모든 FE merge
# ============================================================
FE = (
    meta.merge(hs3_lag_df, on="item_id", how="left")
        .merge(pairs_hs3_best, left_on="item_id", right_on="follow_item", how="left")
        .merge(multi_leader_lag, left_on="item_id", right_on="follower", how="left")
)

FE = FE.drop(columns=["follow_item", "follower"], errors="ignore")

print("🎉 최종 Feature 테이블 생성 완료:", FE.shape)
print(FE.head())




🎉 최종 Feature 테이블 생성 완료: (300, 11)
    item_id  zero_ratio     volatility        cv  hs3  lag      corr lead_hs3  \
0  AANGBULD    0.325581  143585.748259  1.708994  300    3  0.592394      300   
1  AANGBULD    0.325581  143585.748259  1.708994  200    1  0.579046      300   
2  AANGBULD    0.325581  143585.748259  1.708994  481    1  0.556664      300   
3  AHMDUILJ    0.000000   45511.535634  0.358842  600    6  0.409002      600   
4  AHMDUILJ    0.000000   45511.535634  0.358842  720    5  0.404026      600   

   best_lag_hs3  corr_hs3  multi_leader_lag  
0           3.0  0.592394           3.03090  
1           3.0  0.592394           3.03090  
2           3.0  0.592394           3.03090  
3           6.0  0.409002           4.00027  
4           6.0  0.409002           4.00027  


In [11]:

# ============================================================
# 💾 저장
# ============================================================
OUT_PATH = BASE_DIR.parents[1] / "data" / "processed" / "features_basic.csv"
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)
FE.to_csv(OUT_PATH, index=False)

print(f"💾 Feature saved to: {OUT_PATH}")


💾 Feature saved to: /data/ephemeral/home/data/processed/features_basic.csv


In [12]:
# ============================================================
# 9️⃣ pair_train 생성 (baseline 방식)
# ============================================================

import numpy as np
import pandas as pd
from tqdm import tqdm

print("🧩 Create pair_train from pivot_item...")

# pivot_item 다시 생성 (monthly 기반)
pivot_item = monthly.pivot(index="item_id", columns="ym", values="total_value").fillna(0.0)

months = pivot_item.columns.to_list()
n_months = len(months)

def safe_corr(x, y):
    if np.std(x)==0 or np.std(y)==0:
        return 0.0
    return float(np.corrcoef(x, y)[0,1])

pairs = []

for leader in tqdm(pivot_item.index):
    x = pivot_item.loc[leader].values.astype(float)
    
    if np.count_nonzero(x) < 6:
        continue
    
    for follower in pivot_item.index:
        if follower == leader:
            continue
        
        y = pivot_item.loc[follower].values.astype(float)
        if np.count_nonzero(y) < 6:
            continue
        
        best_corr = 0
        best_lag = None
        
        for lag in range(1, 7):
            if n_months <= lag:
                continue
            
            corr = safe_corr(x[:-lag], y[lag:])
            
            if abs(corr) > abs(best_corr):
                best_corr = corr
                best_lag = lag
        
        if best_lag is not None:
            pairs.append([leader, follower, best_lag, best_corr])

pair_df = pd.DataFrame(pairs, columns=[
    "leading_item_id", "following_item_id", "best_lag", "max_corr"
])

print("✔ pair candidates:", pair_df.shape)


# ============================================================
# 🔟 pair_train target 생성
# ============================================================

print("🎯 Generate target (future value)...")

# 마지막 달 = 2025-07
t_last = n_months - 1
t_prev = n_months - 2

# target: follower의 2025-08 value
# → 실제 2025-08은 제공되지 않으므로 submit 때만 예측
# → train은 시계열 내부에서 self-supervised target 생성

rows = []
for row in pair_df.itertuples():
    L = row.leading_item_id
    F = row.following_item_id
    lag = int(row.best_lag)
    corr = float(row.max_corr)
    
    xs = pivot_item.loc[L].values.astype(float)
    ys = pivot_item.loc[F].values.astype(float)
    
    # self-supervised target (b_{t+1})
    for t in range(max(lag,1), n_months-1):
        rows.append({
            "leading_item_id": L,
            "following_item_id": F,
            "b_t": ys[t],
            "b_t_1": ys[t-1],
            "a_t_lag": xs[t-lag],
            "max_corr": corr,
            "best_lag": lag,
            "target": ys[t+1]
        })

pair_train = pd.DataFrame(rows)
print("✔ pair_train:", pair_train.shape)

# 저장
PAIR_PATH = BASE_DIR.parents[1] / "data" / "processed" / "pair_train.csv"
pair_train.to_csv(PAIR_PATH, index=False)
print(f"💾 Saved pair_train to: {PAIR_PATH}")


🧩 Create pair_train from pivot_item...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:04<00:00, 20.62it/s]


✔ pair candidates: (8930, 4)
🎯 Generate target (future value)...
✔ pair_train: (343354, 8)
💾 Saved pair_train to: /data/ephemeral/home/data/processed/pair_train.csv
